In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.preprocessing import normalize

In [2]:
xy = np.loadtxt('C:\\Users\\SANHA\\Desktop\\new_data.csv', delimiter=',', dtype=np.float32)
x_data=xy[:,0:-1]
x_data = normalize(x_data, axis=0, norm='max')
xy[:,0:-1]=x_data
y_data=xy[:,[-1]]
m_data=xy[:499,:]
r_data=xy[:10000,:]
g_data=xy[:0,:]
F_data=xy[:,:]

In [3]:
# Model Inputs
def model_inputs(real_dim, noise_dim):
    inputs_real_ = tf.placeholder(tf.float32, shape=[None, real_dim], name='inputs_real')
    inputs_z_ = tf.placeholder(tf.float32, shape=[None, noise_dim], name='inputs_z')
    
    return inputs_real_, inputs_z_

def leaky_relu(x, alpha):
    return tf.maximum(alpha * x, x)

In [4]:
# Generator Network
def model_generator(z_input, out_dim, n_units=128, reuse=False, alpha=0.01):
    # used to reuse variables, name scope
    with tf.variable_scope('generator', reuse=reuse):
        hidden_layer = tf.layers.dense(z_input, n_units, activation=None)
        hidden_layer = leaky_relu(hidden_layer, alpha)
        
        logits = tf.layers.dense(hidden_layer, out_dim, activation=None)
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits

In [5]:
# Discriminator Network
def model_discriminator(input, n_units=128, reuse=False, alpha=0.1):
    with tf.variable_scope('discriminator', reuse=reuse):
        hidden_layer = tf.layers.dense(input, n_units, activation=tf.nn.relu)
        #hidden_layer = leaky_relu(hidden_layer, alpha)
        
        logits = tf.layers.dense(hidden_layer, 1, activation=None)
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits

In [6]:
#parameter
input_size = 42
z_dim = 21
g_hidden_size = 128
d_hidden_size = 128
alpha = 0.1
smooth = 0.1
learning_rate = 0.001

In [7]:
tf.reset_default_graph()  # If we don't have this, as we call this block over and over, the graph gets bigger and bigger

graph = tf.Graph()
with graph.as_default():
    inputs_real, inputs_z = model_inputs(input_size, z_dim)
    
    g_outputs, g_logits = model_generator(inputs_z, input_size, n_units=g_hidden_size, reuse=False, alpha=alpha)
    
    d_outputs_real, d_logits_real = model_discriminator(inputs_real, n_units=d_hidden_size, reuse=False, alpha=alpha)
    d_outputs_fake, d_logits_fake = model_discriminator(g_outputs, n_units=d_hidden_size, reuse=True, alpha=alpha)
    
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, labels=tf.ones_like(d_logits_real) * (1-smooth)))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.zeros_like(d_logits_fake)))
    
    d_loss = d_loss_real + d_loss_fake
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.ones_like(d_logits_fake)))
    
    t_vars = tf.trainable_variables()
    g_vars = [variable for variable in t_vars if 'generator' in variable.name]
    d_vars = [variable for variable in t_vars if 'discriminator' in variable.name]
    
    # Affected Variables with var_list
    d_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(d_loss, var_list=d_vars)
    g_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(g_loss, var_list=g_vars)
    
    # Saving variables with var_list
    saver = tf.train.Saver(var_list=g_vars)

In [8]:
samples=[]
normal=0
abnormal=0

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    f = open('C:\\Users\\SANHA\\Desktop\\gen_sample3333.txt', 'a+')
    for step in range(1):
        
        
        batch_images = F_data[step].reshape([1, 42])
        batch_z = np.random.uniform(-1, 1, size=[1, z_dim])
        
        _ = sess.run(d_optimizer, feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        _ = sess.run(g_optimizer, feed_dict={inputs_z : batch_z})
        loss_d, loss_g = sess.run([d_loss, g_loss], feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        #if step%1000==0:
        #    print('step {} / {} Complete. D_Loss : {:0.3f}, G_Loss : {:0.3f}'.format(step+1, 100000, loss_d, loss_g))
        sys.stdout.write("\r%d / %d: %f, %f" % (step, 100000, loss_d, loss_g))
        sys.stdout.flush()
        sample_z = np.random.uniform(-1, 1, size=[1, z_dim])  # 16 Samples each epoch
        gen_samples, _ = sess.run(model_generator(inputs_z, input_size, reuse=True), feed_dict={inputs_z : sample_z})
        
        temp=gen_samples[0,41]
        #print(temp)
        if temp>=0.5:
            gen_samples[0,41]=1
            abnormal+=1
        else :
            gen_samples[0,41]=0
            normal+=1
        #print(temp,gen_samples[0,41])
        #write for text to csv
        f.write("%f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f \n" %(gen_samples[0,0],gen_samples[0,1],gen_samples[0,2],gen_samples[0,3],gen_samples[0,4],gen_samples[0,5],gen_samples[0,6],gen_samples[0,7],gen_samples[0,8],gen_samples[0,9],gen_samples[0,10],gen_samples[0,11],gen_samples[0,12],gen_samples[0,13],gen_samples[0,14],gen_samples[0,15],gen_samples[0,16],gen_samples[0,17],gen_samples[0,18],gen_samples[0,19],gen_samples[0,20],gen_samples[0,21],gen_samples[0,22],gen_samples[0,23],gen_samples[0,24],gen_samples[0,25],gen_samples[0,26],gen_samples[0,27],gen_samples[0,28],gen_samples[0,29],gen_samples[0,30],gen_samples[0,31],gen_samples[0,32],gen_samples[0,33],gen_samples[0,34],gen_samples[0,35],gen_samples[0,36],gen_samples[0,37],gen_samples[0,38],gen_samples[0,39],gen_samples[0,40],gen_samples[0,41]))
       
        
    print('Generating Complete. normal={}, abnormal={}'.format(normal,abnormal))
    f.close()

0 / 100000: 1.422688, 0.763870Generating Complete. normal=1, abnormal=0


In [97]:
xy2 = np.genfromtxt('C:\\Users\\SANHA\\Desktop\\mix_data_40000.csv', delimiter=',', dtype=np.float32)
xy3 = np.loadtxt('C:\\Users\\SANHA\\Desktop\\gen_data.csv', delimiter=',', dtype=np.float32)
xy4 = np.loadtxt('C:\\Users\\SANHA\\Desktop\\false_data.csv', delimiter=',', dtype=np.float32)
#gen data 20000
gx_data=xy3[:,0:-1]
gy_data=xy3[:,[-1]]
gF_data=xy3[:,:]

#gen 20000+real 20000
mx_data=xy2[:39999,0:-1]
my_data=xy2[:39999,[-1]]
mF_data=xy2[:39999,:]

#real data 40000
tx_data=r_data[:40000,0:-1]
ty_data=r_data[:40000,[-1]]

#false data
F_data=xy4[:,:]
Fx_data=xy4[:,0:-1]
Fy_data=xy4[:,[-1]]
print("finished")

finished


In [98]:

nb_classes=2

X=tf.placeholder(tf.float32,[None,41])
Y=tf.placeholder(tf.int32,[None,1])

Y_one_hot=tf.one_hot(Y,nb_classes)
Y_one_hot=tf.reshape(Y_one_hot,[-1,nb_classes])

W1=tf.Variable(tf.random_normal([41,41]),name='weight1')
b1=tf.Variable(tf.random_normal([41]),name='bias1')
layer1=tf.sigmoid(tf.matmul(X,W1)+b1)

W2=tf.Variable(tf.random_normal([41,41]),name='weight2')
b2=tf.Variable(tf.random_normal([41]),name='bias2')
layer2=tf.sigmoid(tf.matmul(layer1,W2)+b2)

W3=tf.Variable(tf.random_normal([41,nb_classes]),name='weight3')
b3=tf.Variable(tf.random_normal([nb_classes]),name='bias3')
logits=tf.matmul(layer2,W3)+b3
hypothesis=tf.nn.softmax(logits)

cost_i=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y_one_hot)

cost=tf.reduce_mean(cost_i)
optimizer=tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

prediction=tf.argmax(hypothesis,1) #가능성을 퍼센트로~~
correct_prediction=tf.equal(prediction,tf.arg_max(Y_one_hot,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    f = open('C:\\Users\\SANHA\\Desktop\\false_data.txt', 'a+')
    for step in range(1000):
        sess.run(optimizer,feed_dict={X:x_data,Y:y_data})
        #print("training by gan sample")
        if step %10==0:
            loss,acc=sess.run([cost,accuracy],feed_dict={X:Fx_data,Y:Fy_data})
            print("step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step,loss,acc))
  
    tr=0
    fa=0
    total=0
#m_data=np.append(m_data,gen_samples,axis=0)

    pred = sess.run(prediction, feed_dict={X: x_data})
    for p, y in zip(pred, y_data.flatten()):
            if(p==int(y)):
                tr=tr+1
            else:
                fa=fa+1
                #print(gx_data)
                f.write("%f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f \n" %(gx_data[fa,0],gx_data[fa,1],gx_data[fa,2],gx_data[fa,3],gx_data[fa,4],gx_data[fa,5],gx_data[fa,6],gx_data[fa,7],gx_data[fa,8],gx_data[fa,9],gx_data[fa,10],gx_data[0,11],gx_data[fa,12],gx_data[fa,13],gx_data[fa,14],gx_data[fa,15],gx_data[fa,16],gx_data[fa,17],gx_data[fa,18],gx_data[fa,19],gx_data[fa,20],gx_data[fa,21],gx_data[fa,22],gx_data[fa,23],gx_data[fa,24],gx_data[fa,25],gx_data[fa,26],gx_data[fa,27],gx_data[fa,28],gx_data[fa,29],gx_data[fa,30],gx_data[fa,31],gx_data[fa,32],gx_data[fa,33],gx_data[fa,34],gx_data[fa,35],gx_data[fa,36],gx_data[fa,37],gx_data[fa,38],gx_data[fa,39],gx_data[fa,40],gy_data[fa,0]))
                
            #print("[{}] Prediction: {} Real Y: {}".format(p == int(y), p, int(y)))
    f.close()
    print("true={} false: {} acc: {:0.2f}".format(tr,fa,tr/(tr+fa)))

step:     0	Loss: 0.669	Acc: 66.50%
step:    10	Loss: 0.832	Acc: 58.92%
step:    20	Loss: 1.035	Acc: 60.39%
step:    30	Loss: 1.265	Acc: 59.17%
step:    40	Loss: 1.432	Acc: 58.56%
step:    50	Loss: 1.501	Acc: 57.58%
step:    60	Loss: 1.508	Acc: 57.46%
step:    70	Loss: 1.517	Acc: 57.70%
step:    80	Loss: 1.567	Acc: 58.19%
step:    90	Loss: 1.636	Acc: 59.90%
step:   100	Loss: 1.689	Acc: 61.98%
step:   110	Loss: 1.732	Acc: 63.08%
step:   120	Loss: 1.771	Acc: 64.18%
step:   130	Loss: 1.835	Acc: 64.30%
step:   140	Loss: 1.864	Acc: 64.91%
step:   150	Loss: 1.913	Acc: 65.89%
step:   160	Loss: 1.944	Acc: 66.38%
step:   170	Loss: 1.972	Acc: 66.75%
step:   180	Loss: 1.999	Acc: 67.97%
step:   190	Loss: 2.016	Acc: 68.46%
step:   200	Loss: 2.032	Acc: 68.95%
step:   210	Loss: 2.056	Acc: 69.80%
step:   220	Loss: 2.078	Acc: 70.17%
step:   230	Loss: 2.089	Acc: 71.15%
step:   240	Loss: 2.114	Acc: 71.15%
step:   250	Loss: 2.109	Acc: 71.52%
step:   260	Loss: 2.116	Acc: 71.88%
step:   270	Loss: 2.119	Acc:

KeyboardInterrupt: 

In [101]:
with tf.Session() as sess:  
    pred = sess.run(prediction, feed_dict={X: x_data})
    for p, y in zip(pred, y_data.flatten()):
            if(p==int(y)):
                tr=tr+1
            else:
                fa=fa+1
                #print(gx_data)
                f.write("%f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f \n" %(gx_data[fa,0],gx_data[fa,1],gx_data[fa,2],gx_data[fa,3],gx_data[fa,4],gx_data[fa,5],gx_data[fa,6],gx_data[fa,7],gx_data[fa,8],gx_data[fa,9],gx_data[fa,10],gx_data[0,11],gx_data[fa,12],gx_data[fa,13],gx_data[fa,14],gx_data[fa,15],gx_data[fa,16],gx_data[fa,17],gx_data[fa,18],gx_data[fa,19],gx_data[fa,20],gx_data[fa,21],gx_data[fa,22],gx_data[fa,23],gx_data[fa,24],gx_data[fa,25],gx_data[fa,26],gx_data[fa,27],gx_data[fa,28],gx_data[fa,29],gx_data[fa,30],gx_data[fa,31],gx_data[fa,32],gx_data[fa,33],gx_data[fa,34],gx_data[fa,35],gx_data[fa,36],gx_data[fa,37],gx_data[fa,38],gx_data[fa,39],gx_data[fa,40],gy_data[fa,0]))
                
            #print("[{}] Prediction: {} Real Y: {}".format(p == int(y), p, int(y)))

FailedPreconditionError: Attempting to use uninitialized value bias3_43
	 [[Node: bias3_43/read = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](bias3_43)]]

Caused by op 'bias3_43/read', defined at:
  File "C:\Users\SANHA\Anaconda3\envs\py35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\SANHA\Anaconda3\envs\py35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\SANHA\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\SANHA\Anaconda3\envs\py35\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\SANHA\Anaconda3\envs\py35\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Users\SANHA\Anaconda3\envs\py35\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_for